In [1]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
import os
from tqdm import trange

from pathlib import Path
from glob import glob


Autosaving every 180 seconds


In [2]:

import os
os.environ["DLClight"] = "True"
import deeplabcut
import tensorflow.contrib.slim as slim
from pathlib import Path
import numpy as np
import tensorflow as tf
vers = (tf.__version__).split(".")
if int(vers[0]) == 1 and int(vers[1]) > 12:
    TF = tf.compat.v1
else:
    TF = tf
from tensorflow.contrib.slim.nets import resnet_v1
from tensorflow.keras.datasets import cifar10
#from deeplabcut.pose_estimation_tensorflow.train import get_optimizer, LearningRate
from deeplabcut.pose_estimation_tensorflow.train import  LearningRate


DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


In [3]:
#
# ------------------- User inputs --------
# define paths

def get_optimizer(loss_op, cfg, train_resnet=False):
    # train_resnet=True trains resnet!
    learning_rate = TF.placeholder(tf.float32, shape=[])

    if cfg.optimizer == "sgd":
        optimizer = TF.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
    elif cfg.optimizer == "adam":
        optimizer = TF.train.AdamOptimizer(learning_rate)
    else:
        raise ValueError('unknown optimizer {}'.format(cfg.optimizer))

    #%%
    all_variables_to_train = tf.trainable_variables()
    if train_resnet:
        variables_to_train = all_variables_to_train
    else:
        variables_to_train = list(filter(lambda k: 'mclassid' in k.name, all_variables_to_train))

    train_op = slim.learning.create_train_op(loss_op, optimizer, variables_to_train=variables_to_train)

    return learning_rate, train_op

#
def get_train_config(cfg, shuffle=1):
    from deeplabcut.utils import auxiliaryfunctions
    from deeplabcut.pose_estimation_tensorflow.config import load_config
    project_path = cfg['project_path']
    iteration = cfg['iteration']
    TrainingFraction = cfg['TrainingFraction'][iteration]
    modelfolder = os.path.join(
        project_path,
        str(auxiliaryfunctions.GetModelFolder(TrainingFraction, shuffle, cfg)))

    path_test_config = Path(modelfolder) / 'train' / 'pose_cfg.yaml'
    print(path_test_config)
    try:
        dlc_cfg = load_config(str(path_test_config))
    except FileNotFoundError:
        raise FileNotFoundError(
            "It seems the model for shuffle %s and trainFraction %s does not exist."
            % (shuffle, TrainingFraction))
    return dlc_cfg




In [4]:


config_path = '/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/dlc/config.yaml'
videofile_path = '/media/cat/4TBSSD/DLC_full_directory/videos_to_label/2020-3-16_12_54_07_193951_compressed_label.mp4'
shuffle = 1 #dlc experiment shuffle

#
# train w imagenet: data/255
#BATCH_SIZE = 60
num_classes = 4
is_training=False

In [5]:
# LOAD GERBIL DATA SPECIFICALLY FORMATED
#%%

def load_gerbil_training_data_34_x_34(animal_dir, 
                              n_networks):
    
    from skimage.measure import block_reduce
    
    x_train = []
    y_train = []
    x_test = []
    y_test = []
    for k in range(n_networks):
        data = np.load(animal_dir+str(k)+'.npy')
        
        # scale data down:
        #image = np.arange(3*3*4).reshape(3, 3, 4)
        data_downsampled = []
        for p in range(data.shape[0]):
            temp = data[p]
           # print ("temp start: ", temp.shape)
            temp = block_reduce(data[p], 
                                block_size=(6,6), 
                                func=np.mean)
            #print ("temp finish: ", temp.shape)
            data_downsampled.append(temp)
        data = np.array(data_downsampled)
        
        idx_rand = np.random.choice(np.arange(data.shape[0]), int(data.shape[0]*0.8), replace=False)
        #print ("idx rand: ", idx_rand.shape)
        
        x_train.append(data[idx_rand])
        print (data[idx_rand].shape)
        y_train.append(np.zeros(idx_rand.shape[0])+k)

        # make test set
        idx_test = np.delete(np.arange(data.shape[0]), 
                             idx_rand)
        #print ("idx test: ", idx_test.shape)
        temp = data[idx_test]
        #print (temp.shape)
        x_test.append(temp)
        y_test.append(np.zeros(temp.shape[0])+k)

        

    x_train = np.vstack(x_train)
    x_train = np.int32((x_train,x_train,x_train)).transpose(1,2,3,0)
    #np.save('/home/cat/x_train.npy', x_train)
    y_train = np.hstack(y_train)#[:,None]
    
    x_test = np.vstack(x_test)
    x_test = np.int32((x_test,x_test,x_test)).transpose(1,2,3,0)
    y_test = np.hstack(y_test) 
        
        
    return x_train, y_train, x_test, y_test 

def load_gerbil_training_data_100_x_100(animal_dir, 
                              n_networks):
    
    from skimage.measure import block_reduce
    
    x_train = []
    y_train = []
    x_test = []
    y_test = []
    for k in range(n_networks):
        data = np.load(animal_dir+str(k)+'.npy')
        
        # scale data down:
        #image = np.arange(3*3*4).reshape(3, 3, 4)
        data_downsampled = []
        for p in range(data.shape[0]):
            temp = data[p]
           # print ("temp start: ", temp.shape)
            temp = block_reduce(data[p], 
                                block_size=(2,2), 
                                func=np.mean)
            #print ("temp finish: ", temp.shape)
            data_downsampled.append(temp)
        data = np.array(data_downsampled)
        
        idx_rand = np.random.choice(np.arange(data.shape[0]), int(data.shape[0]*0.8), replace=False)
        #print ("idx rand: ", idx_rand.shape)
        
        x_train.append(data[idx_rand])
        print (data[idx_rand].shape)
        y_train.append(np.zeros(idx_rand.shape[0])+k)

        # make test set
        idx_test = np.delete(np.arange(data.shape[0]), 
                             idx_rand)
        #print ("idx test: ", idx_test.shape)
        temp = data[idx_test]
        #print (temp.shape)
        x_test.append(temp)
        y_test.append(np.zeros(temp.shape[0])+k)

        

    x_train = np.vstack(x_train)
    x_train = np.int32((x_train,x_train,x_train)).transpose(1,2,3,0)
    #np.save('/home/cat/x_train.npy', x_train)
    y_train = np.hstack(y_train)#[:,None]
    
    x_test = np.vstack(x_test)
    x_test = np.int32((x_test,x_test,x_test)).transpose(1,2,3,0)
    y_test = np.hstack(y_test) 
        
        
    return x_train, y_train, x_test, y_test 


animal_dir ='/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/training_images_cc/segments_training/'
#animal_dir = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/training_images/animals/'
animal_dir ='/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/2020-3-9_12_14_22_815059_compressed/cnn_training/animals/'


def make_training_arrays(animal_dir):
    
    import os
    dirs = os.listdir(animal_dir)
    
    if len(dirs)>4:
        return
    
    print(dirs)
    for ctr,dir_ in enumerate(dirs):
        fnames = os.listdir(animal_dir+'/'+dir_)
        
        imgs = []
        for fname in fnames:
            
            imgs.append(np.load(
                            os.path.join(animal_dir,
                                         dir_, fname                           
                                            )))
        imgs = np.array(imgs)
        np.save(os.path.join(animal_dir,
                            str(ctr)),
               imgs)
        print (imgs.shape)
    
make_training_arrays(animal_dir)

n_networks = 4

(x_train, y_train, x_test, y_test) = load_gerbil_training_data_100_x_100(animal_dir,
                                                              n_networks)

nx_in, ny_in, nc_in = x_train.shape[1:]
assert nc_in == 3

print ("x train: ", x_train.shape, 
       "y train: ", y_train.shape,  
       "x test: ", x_test.shape, 
       "y test: ", y_test.shape)

# 4-fold training; 
# required dimensions: [#_frames x 4, 100, 100, 3] ;  output [# frames x 4]

(2032, 100, 100)
(2137, 100, 100)
(1932, 100, 100)
(2328, 100, 100)
x train:  (8429, 100, 100, 3) y train:  (8429,) x test:  (2110, 100, 100, 3) y test:  (2110,)


In [6]:
# ------------------- READ DLC info --------
# read config
print("Reading config...")
cfg = deeplabcut.auxiliaryfunctions.read_config(str(config_path))

Reading config...


In [7]:
# ------------------- READ DLC snapshot --------

#overwrite_snapshot = None # load  resnet weights trained until last dlc snapshot
# set to 'snapshot-0' to load resnet weights not trained for tracking w dlc
#
def get_train_config(cfg, shuffle=1):
    from deeplabcut.utils import auxiliaryfunctions
    from deeplabcut.pose_estimation_tensorflow.config import load_config
    project_path = cfg['project_path']
    iteration = cfg['iteration']
    TrainingFraction = cfg['TrainingFraction'][iteration]
    modelfolder = os.path.join(
        project_path,
        str(auxiliaryfunctions.GetModelFolder(TrainingFraction, shuffle, cfg)))

    path_test_config = Path(modelfolder) / 'train' / 'pose_cfg.yaml'
    print(path_test_config)
    try:
        dlc_cfg = load_config(str(path_test_config))
    except FileNotFoundError:
        raise FileNotFoundError(
            "It seems the model for shuffle %s and trainFraction %s does not exist."
            % (shuffle, TrainingFraction))
    return dlc_cfg

dlc_cfg = get_train_config(cfg,shuffle=shuffle)

/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/dlc/dlc-models/iteration-0/madeline_july2Jul2-trainset95shuffle1/train/pose_cfg.yaml


In [8]:
#%% ------------- BEGIN GRAPH -------------
TF.reset_default_graph()
# tf.data API
batch_size = tf.placeholder(tf.int64, shape=[])
x, y = tf.placeholder(tf.float32, shape=[None, nx_in, ny_in, nc_in]), tf.placeholder(tf.int32, shape=[None,])
train_dataset = tf.data.Dataset.from_tensor_slices((x, y)).batch(batch_size).repeat().shuffle(5000)
test_dataset = tf.data.Dataset.from_tensor_slices((x, y)).batch(batch_size).shuffle(5000)
iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                           train_dataset.output_shapes)
image_batch, targets = iterator.get_next()


In [9]:

# init iterators
train_init_op = iterator.make_initializer(train_dataset)
test_init_op = iterator.make_initializer(test_dataset)
# upsample cifar images which are 32 x 32
if nx_in <= 200:
    inputs = tf.image.resize_images(image_batch, [224, 224])
targets = tf.reshape(targets, [batch_size, ])


In [10]:

# Make model
net_funcs = {'resnet_50': resnet_v1.resnet_v1_50,
             'resnet_101': resnet_v1.resnet_v1_101,
             'resnet_152': resnet_v1.resnet_v1_152}

net_fun = net_funcs[dlc_cfg.net_type]
# preprocess the same way as dlc -- here note that they don't normalize
mean = tf.constant(dlc_cfg.mean_pixel, dtype=tf.float32, shape=[1, 1, 1, 3],
                   name='img_mean')
im_centered = inputs - mean
# add strides st we can load dlc weights
if 'output_stride' not in dlc_cfg.keys():
    dlc_cfg.output_stride = 16
if 'deconvolutionstride' not in dlc_cfg.keys():
    dlc_cfg.deconvolutionstride = 2
with slim.arg_scope(resnet_v1.resnet_arg_scope()):
    net, endpoints = net_fun(im_centered, num_classes=None,#num_classes,
                             is_training=is_training,
                             #global_pool=True,
                             output_stride=dlc_cfg.output_stride)

net = tf.squeeze(net, axis=[1, 2])
logits = slim.fully_connected(net, num_outputs=num_classes,
                              activation_fn=None,
                              scope='mclassid')
loss = tf.reduce_mean(
    tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, 
                                                   labels=targets))

In [11]:
# add extrainfo
logits2 = tf.nn.softmax(logits)
classes = tf.cast(tf.argmax(logits2, axis=1, ), tf.int32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(classes, targets), dtype=tf.float32))
eval_dict = {'logits2': logits2,
             'classes': classes,
             'accuracy': accuracy}

In [12]:

# restore variables for resnet
variables_to_restore = slim.get_variables_to_restore(include=["resnet_v1"])
restorer = TF.train.Saver(variables_to_restore)
saver = TF.train.Saver(
        max_to_keep=5
    )

In [13]:
# %% Init session
print ("1")
config_TF = TF.ConfigProto()
config_TF.gpu_options.allow_growth = True
sess = TF.Session(config=config_TF)
TF.summary.FileWriter('logs/', sess.graph)
print ("2")

learning_rate, train_op = get_optimizer(loss, dlc_cfg)
lr = tf.get_variable('learning_rate', initializer=0.01, trainable=False)
print ("3")
#%%
sess.run(TF.global_variables_initializer())
print ("4")
sess.run(TF.local_variables_initializer())
print ("DONE")

1
2
3
4
DONE


In [15]:
# Restore the resnet weights from dlc
dlc_cfg.init_weights = '/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/dlc/resnet_v1_50_2016_08_28/resnet_v1_50.ckpt'

#model_name = 
#dlc_cfg.init_weights = '/home/cat/code/dlc_data/snapshots/mclass_epoch49-iter96--96'

restorer.restore(sess, dlc_cfg.init_weights)
print('Restored variables from\n{}\n'.format(dlc_cfg.init_weights))

Restored variables from
/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/dlc/resnet_v1_50_2016_08_28/resnet_v1_50.ckpt



In [ ]:
#%%
lr_gen = LearningRate(dlc_cfg)
#%%
EPOCHS = 50 # 10000
current_lr = 0.0001
BATCH_SIZE = 300
n_batches = max(1, int(x_train.shape[0] / BATCH_SIZE))
if True:
    print('Training...')
    for epoch in range(EPOCHS):
        sess.run(train_init_op, feed_dict={x: x_train, y: y_train, batch_size: BATCH_SIZE,
                                           learning_rate: current_lr})

        tot_loss = 0
        for iter in range(n_batches):
            _, loss_value, accuracy_value = sess.run([train_op, loss, accuracy],
                                                     feed_dict={learning_rate:current_lr})
            if iter%25==0:
                print("Epoch: {} \t Iter: {}/{}, Loss: {:.4f} Accuracy:{:.4f}".format(epoch, iter, n_batches, loss_value, accuracy_value))
            tot_loss += loss_value

            if iter % 2500 == 0 or (iter+1 == n_batches):
                model_name = 'snapshots/mclass_epoch{}-iter{}-'.format(epoch,iter)
                saver.save(sess, model_name, global_step=iter)
                if iter +1  == EPOCHS:
                    model_name = 'snapshots/mclass_epoch{}-iter{}-final-'.format(epoch,iter)
                    saver.save(sess, model_name, global_step=0)

        print("Epoch: {}, Loss: {:.4f}".format(epoch, tot_loss / n_batches))
        # initialise iterator with test data
        sess.run(test_init_op,
                 feed_dict={x: x_test, y: y_test, batch_size: BATCH_SIZE, learning_rate:current_lr})
        print('Epoch: {}, '.format(epoch) + 'Test Loss: {:.4f} Test Accuracy {:.4f}'.format(*sess.run([loss,accuracy])))

else:

    model_name = '/home/cat/code/dlc_data/snapshots/mclass_epoch49-iter96--96'
    dlc_cfg.init_weights = model_name

    restorer.restore(sess, dlc_cfg.init_weights)
    print('Restored variables from\n{}\n'.format(dlc_cfg.init_weights))


Training...


In [18]:
# #%% TEST ON THE TRAINING DATA
# batch_np = 1
# print (x_test.shape)
# #for ii ,(image_np, target_np) in enumerate(zip(x_test, y_test)):
# ids = np.random.choice(np.arange(x_test.shape[0]), 500, replace=False)
# ctr=0
# for id_ in ids:
#     image_np = x_test[id_][None, :]
#     target_np = y_test[id_]
#     feed_dict ={image_batch: image_np,
#                 #targets: target_np,
#                 batch_size: batch_np}

#     classes_np, logits2_np = sess.run([classes, logits2], feed_dict=feed_dict)
#     if id_%100==0:
#         print ("image: ", id_, "  predicted: ", classes_np[0], " true val: ", target_np, "  confidence: ", logits2_np.max())
#         print ('...')
#     if classes_np[0]==target_np:
#         ctr+=1
#     #p#rint('True class {} Predicted class {} Confidence {}'.format(target_np, classes_np[0], logits2_np.max()))
# #%%
# print ("% Correct: ", ctr/ids.shape[0])


In [19]:
def predict_cnn2(x_test, 
                y_test):
    
    batch_np = 10
    ids = np.arange(0,
                    x_test.shape[0],
                    batch_np)

    ctr=0
    class_array = []
    logit_array = []
    for id_ in tqdm(ids):

        #print ("frame: ", id_)
        image_np = x_test[id_:id_+batch_np] #[None, :]
        target_np = y_test[id_:id_+batch_np]
        feed_dict ={image_batch: image_np,
                    #targets: target_np,
                    batch_size: batch_np}

        classes_np, logits2_np = sess.run([classes, logits2], feed_dict=feed_dict)
        #if classes_np[0]==target_np:
        #    ctr+=1
        class_array.append(classes_np)
        logit_array.append(np.max(logits2))
        
    return ctr, ids.shape[0], class_array, logit_array


def load_gerbil_single_directory_100_x_100(animal_dir,
                                 animal_id):
    
    from skimage.measure import block_reduce
    import glob
    
    data = []
    #print(animal_dir+'/**/*.npy')
    fnames = glob.glob(animal_dir+'*.npy',recursive = True)
    #print ("Fnames; ", len(fnames))
    
    ctr_k = 0
    frame_ids = []
    for ctr, fname in enumerate(fnames):
        if '.npy' in fname:
            temp = np.load(fname)
            idx1 = fname.index('frame_')
            idx2 = fname.index('_id_')
            frame_id = int(fname[idx1+6:idx2])
            #print (frame_id)
            frame_ids.append(frame_id)
            if temp.shape[0]!= 200 or temp.shape[1]!=200:
                continue
            #print (ctr, temp.shape)
            data.append(temp)
            ctr_k+=1
    #print (ctr_k)
            
    data = np.array(data)
    #print ("data: ", 
    #       data.shape)

    x_test = []
    y_test = []
    #data = np.load(animal_dir+"id_"+str(k)+'.npy')
        
    # scale data down:
    #image = np.arange(3*3*4).reshape(3, 3, 4)
    data_downsampled = []
    for p in range(data.shape[0]):
        temp = data[p]
        #print ("temp start: ", temp.shape)
        temp = block_reduce(temp, 
                            block_size=(2,2), 
                            func=np.mean)
        #print ("temp finish: ", temp.shape)
        data_downsampled.append(temp)
    x_test = np.array(data_downsampled)
    #print (x_test.shape)
    y_test = np.zeros(x_test.shape[0])+ animal_id
    
    #x_test = np.vstack(x_test)
    x_test = np.int32((x_test,x_test,x_test)).transpose(1,2,3,0)
    y_test = np.hstack(y_test)[:,None]
        
    #print ("x test: ", x_test.shape,
    #       "y test: ", y_test.shape)
    
    return x_test, y_test, frame_ids


    

In [20]:

######################################################################
############## ALL DATA FORMATTING FOR CLASSIFICATION ################
######################################################################

# MAKE TRAINING DATA FOR A SINGLE NETWORK
from pathlib import Path
import glob

root_dirs = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/training_images_cc/segments_all/'
root_dirs = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/training_images/'
dirs = glob.glob(root_dirs + '/**')

x_array = []
y_array = []
frames_array = []
vals = []
x_array = np.zeros((300000, 100, 100, 3),'uint8')
from tqdm import tqdm
start = 0
for ctr, dir_ in enumerate(tqdm(dirs)):
    # print (dir_)
    (x_test, y_test, frame_ids) = load_gerbil_single_directory_100_x_100(dir_+'/', ctr)
    #print (x_test.shape, y_test.shape)
    frames_array.append(frame_ids)
    
    #x_array.append(x_test)
    length = x_test.shape[0]
    x_array[start:start+length]= x_test
    #print (start, length)
    start = start+ length    
    y_array.append(y_test)
    vals.append(np.zeros(x_test.shape[0])+np.int32(os.path.split(dir_)[1]))
    
vals = np.hstack(vals)
#print (vals)
#x_test = np.vstack(x_array)
x_test = x_array[:start]
print (x_test.shape)
y_test = np.vstack(y_array)
print (y_test.shape)


100%|██████████| 12467/12467 [05:58<00:00, 34.74it/s] 


IndexError: index 292353 is out of bounds for axis 1 with size 100

In [23]:
print (x_array.shape)
x_test = x_array[:start]
print (x_test.shape)
y_test = np.vstack(y_array)
print (y_test.shape)

(300000, 100, 100, 3)
(292353, 100, 100, 3)
(292353, 1)


In [ ]:
# np.save('/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/training_images_cc/segments_all/frames_array.npy',
#        frames_array)


In [24]:
######################################################################
############## RUN CLASSIFIER ON ALL DATA ############################
######################################################################
(ctr, ids, classes, logits) = predict_cnn2(x_test, y_test)
  
classes = np.hstack(classes)
print ("classes: ", classes)

np.savez(root_dirs+'/classification_output.npz',
        x_test=x_test,
        y_test=y_test,
        vals=vals,
        classes=classes,
        frames_array=frames_array)
#print (np.unique(classes, return_counts=True))
    
    

100%|██████████| 29236/29236 [46:40<00:00, 10.44it/s]  


classes:  [4 4 4 ... 0 0 0]


In [ ]:
print ("CAN not do any more visualization, backend is off : GO TO NOTEBOOK 12: MOVIE MAKING")

In [24]:
# ######################################################################
# ############## SHOW EXAMPLES OF CLASSIFIED DATA ######################
# ######################################################################
# start = 0
# end = 24
# vals_temp = vals[start:end]

# data = np.load('/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/training_images_cc/segments_all/classification_output.npz')

# print (np.unique(vals_temp).shape)
# for ctr, dir_ in enumerate(np.unique(vals_temp)):
#     idx = np.where(vals_temp==dir_)
    
#     temp_classes = classes[idx]
    
#     ax=plt.subplot(4,6,ctr+1)
#     print (dir_, temp_classes)
# plt.show
    
    


In [25]:
######################################################################
# ############## SHOW DISTRIBUTION OF CLASSSES ######################
# ######################################################################

# classes = np.hstack(classes)
# print (classes)

# y = np.histogram(classes, np.arange(5))
# plt.bar(y[1][:-1], y[0], 0.9)
# plt.show()

In [1]:
# # 

# def load_gerbil_all_data2(fname_test_data,
#                         animal_id):
    
#     ''' This loads previously saved data arrays from 4 animals
    
#     '''
#     from skimage.measure import block_reduce
    
#     data = np.load(fname_test_data)

#     x_test = []
#     y_test = []
#     #data = np.load(animal_dir+"id_"+str(k)+'.npy')
        
#     # scale data down:
#     #image = np.arange(3*3*4).reshape(3, 3, 4)
#     data_downsampled = []
#     for p in range(data.shape[0]):
#         temp = data[p]
#        # print ("temp start: ", temp.shape)
#         temp = block_reduce(data[p], 
#                             block_size=(6,6), 
#                             func=np.mean)
#         #print ("temp finish: ", temp.shape)
#         data_downsampled.append(temp)
#     x_test = np.array(data_downsampled)
#     #print (x_test.shape)
#     y_test = np.zeros(x_test.shape[0])+ animal_id
    
#     #x_test = np.vstack(x_test)
#     x_test = np.int32((x_test,x_test,x_test)).transpose(1,2,3,0)
#     y_test = np.hstack(y_test)[:,None]
        
#     print ("x test: ", x_test.shape,
#            "y test: ", y_test.shape)
#     return x_test, y_test


# def load_gerbil_all_data(animal_dir,
#                         animal_id):
    
#     from skimage.measure import block_reduce
    
#     names = ['female','male','pup1','pup2']

#     import glob

    
#     if True:
#         data = []
#         fnames = glob.glob(animal_dir+'/**/*',recursive = True)
#         #print ("Fnames; ", fnames)
#         for fname in fnames:
#             if '.npy' in fname:
#                 temp = np.load(fname)
#                 print (temp.shape)
#                 data.append(temp)
                
#         data = np.array(data)
#         print ("data: ", 
#                data.shape)

#     x_test = []
#     y_test = []
#     #data = np.load(animal_dir+"id_"+str(k)+'.npy')
        
#     # scale data down:
#     #image = np.arange(3*3*4).reshape(3, 3, 4)
#     data_downsampled = []
#     for p in range(data.shape[0]):
#         temp = data[p]
#         print ("temp start: ", temp.shape)
#         temp = block_reduce(temp, 
#                             block_size=(6,6), 
#                             func=np.mean)
#         #print ("temp finish: ", temp.shape)
#         data_downsampled.append(temp)
#     x_test = np.array(data_downsampled)
#     print (x_test.shape)
#     y_test = np.zeros(x_test.shape[0])+ animal_id
    
#     #x_test = np.vstack(x_test)
#     x_test = np.int32((x_test,x_test,x_test)).transpose(1,2,3,0)
#     y_test = np.hstack(y_test)[:,None]
        
#     print ("x test: ", x_test.shape,
#            "y test: ", y_test.shape)
#     return x_test, y_test


# animals = ['female','male','pup1','pup2']

# if False:
#     root_dir = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/training_images/animals/'

#     for animal_id, animal in enumerate(animals):
#         animal_dir = root_dir + animal

#         (x_test, y_test) = load_gerbil_all_data(animal_dir,
#                             animal_id)

        
#         (ctr, n_images, classes_array, logit_array) = predict_cnn(x_test, 
#                                                                   y_test)

# if False:
#     root_dir = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/training_images/animals/id_'
#     for animal_id in range(4):
        
#         fname_test_data = root_dir + str(animal_id) + '_test.npy'
        
#         #if animal_id ==1:
#         #    continue
            
#         (x_test, y_test) =  load_gerbil_all_data2(fname_test_data,
#                         animal_id)
        
#         (ctr, n_images, classes_array, logit_array) = predict_cnn(x_test, 
#                                                                   y_test)
        
#         print (animals[animal_id], animal_id,
#                " # images: ", n_images, 
#                " correct classification: ", ctr/n_images, "%")

In [2]:
# #%% NEW: load one frame
# import tqdm

# def predict_cnn(x_test, 
#                 y_test):
    
#     batch_np = 10
#     ids = np.arange(0,
#                     x_test.shape[0],
#                     batch_np)

#     ctr=0
#     class_array = []
#     logit_array = []
#     for id_ in ids:
#         #if id_%100==0:
#         print ("fraem: ", id_)
#         image_np = x_test[id_:id_+batch_np] #[None, :]
#         target_np = y_test[id_:id_+batch_np]
#         feed_dict ={image_batch: image_np,
#                     #targets: target_np,
#                     batch_size: batch_np}

#         classes_np, logits2_np = sess.run([classes, logits2], feed_dict=feed_dict)
#         #if classes_np[0]==target_np:
#         #    ctr+=1
#         class_array.append(classes_np)
#         logit_array.append(np.max(logits2))
        
#     return ctr, ids.shape[0], class_array, logit_array



In [3]:
# 
# def load_gerbil_single_directory(animal_dir,
#                                  animal_id):
    
#     from skimage.measure import block_reduce
    
#     import glob

    
#     data = []
#     #print(animal_dir+'/**/*.npy')
#     fnames = glob.glob(animal_dir+'/**/*.npy',recursive = True)
#     print ("Fnames; ", len(fnames))
    
#     ctr_k = 0
#     for ctr, fname in enumerate(fnames):
#         if '.npy' in fname:
#             temp = np.load(fname)
#             if temp.shape[0]!= 200 or temp.shape[1]!=200:
#                 continue
#             #print (ctr, temp.shape)
#             data.append(temp)
#             ctr_k+=1
#     print (ctr_k)
            
#     data = np.array(data)
#     print ("data: ", 
#            data.shape)

#     x_test = []
#     y_test = []
#     #data = np.load(animal_dir+"id_"+str(k)+'.npy')
        
#     # scale data down:
#     #image = np.arange(3*3*4).reshape(3, 3, 4)
#     data_downsampled = []
#     for p in range(data.shape[0]):
#         temp = data[p]
#         #print ("temp start: ", temp.shape)
#         temp = block_reduce(temp, 
#                             block_size=(6,6), 
#                             func=np.mean)
#         #print ("temp finish: ", temp.shape)
#         data_downsampled.append(temp)
#     x_test = np.array(data_downsampled)
#     print (x_test.shape)
#     y_test = np.zeros(x_test.shape[0])+ animal_id
    
#     #x_test = np.vstack(x_test)
#     x_test = np.int32((x_test,x_test,x_test)).transpose(1,2,3,0)
#     y_test = np.hstack(y_test)[:,None]
        
#     print ("x test: ", x_test.shape,
#            "y test: ", y_test.shape)
#     return x_test, y_test

# # MAKE TRAINING DATA FOR A SINGLE NETWORK

# # root_dir = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/training_images/10000_frames_all_networks/5/'
# # animal_id = 0

# # (x_test, y_test) = load_gerbil_single_directory(root_dir,
# #                                                 animal_id)
# # #print (x_test.shape)

# # print ("DONE Loading data")



In [18]:
# def predict_cnn2(x_test, 
#                 y_test):
    
#     batch_np = 10
#     ids = np.arange(0,
#                     x_test.shape[0],
#                     batch_np)

#     ctr=0
#     class_array = []
#     logit_array = []
#     for id_ in ids:
#         #if id_%100==0:
#         print ("fraem: ", id_)
#         image_np = x_test[id_:id_+batch_np] #[None, :]
#         target_np = y_test[id_:id_+batch_np]
#         feed_dict ={image_batch: image_np,
#                     #targets: target_np,
#                     batch_size: batch_np}

#         classes_np, logits2_np = sess.run([classes, logits2], feed_dict=feed_dict)
#         #if classes_np[0]==target_np:
#         #    ctr+=1
#         class_array.append(classes_np)
#         logit_array.append(np.max(logits2))
        
#     return ctr, ids.shape[0], class_array, logit_array


# root_dir = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/training_images/10000_frames_all_networks/'


# current_dir = root_dir
# all_sub_dir_paths = glob(str(current_dir) + '/*/') # returns list of sub directory paths

# all_sub_dir_names = [Path(sub_dir).name for sub_dir in all_sub_dir_paths] 

# print (len(all_sub_dir_names))

# for dir_ in all_sub_dir_names[10:11]:
#     (x_test, y_test) = load_gerbil_single_directory(root_dir+dir_,
#                                                     0)
#     #print (type(x_test))
    
#     (ctr, ids, classes, logits) = predict_cnn2(x_test,
#                                               y_test)
    
#     # 
#     print ("classes: ", classes)
#     print (np.unique(classes, return_counts=True))
#     #break
    

In [38]:
# 

#image_batch = x_test[:10]
#inputs = tf.image.resize_images(image_batch, [224, 224])
#print (inputs.shape)
graph = tf.get_default_graph()
    
feed_dict={x: x_test[:10]}

##model = graph.get_tensor_by_name('resnet_model/final_dense:0')
#model.predict()
model = graph.get_tensor_by_name('/home/cat/code/dlc_toolbox/snapshots/mclass_epoch1-iter32--32.data-00000-of-00001')

#res = sess.run(model, {inputs:img})
    
#res = sess.predict(test_init_op,  
#               feed_dict = feed_dict)




ValueError: The name '/home/cat/code/dlc_toolbox/snapshots/mclass_epoch1-iter32--32.data-00000-of-00001' looks like an (invalid) Operation name, not a Tensor. Tensor names must be of the form "<op_name>:<output_index>".